In [1]:
# run this to shorten the data import from the files
import os
cwd = os.path.dirname(os.getcwd())+'/'
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [4]:
import pandas as pd

ratings = pd.read_csv(path_data+'googleplaystore.csv')
ratings.head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up


In [5]:
# exercise 01

"""
Exploring Google apps data

The first dataset you'll work with contains information about the ratings of around 10,800 apps on the Google Play store. It has been preloaded into a DataFrame called ratings. There are 13 features that describe a given app, such as 'Category' and 'Price'. The goal is to use this information to predict the rating of an app, which can range from 1 to 5.

Before doing that, it's a good idea to familiarize yourself with the dataset using pandas methods such as .head() and .describe(). Explore the data in the IPython Shell and select the incorrect statement from the options below.
"""

# Instructions

"""
Possible answers:
    
    The average rating is around 3.62.
    
    The median rating is greater than the mean.
    
    All the ratings lie in the range from 0 to 5. {Answer}
    
    Most of the ratings are 3 or higher.
    
    'Rating' is the only numerical feature.
"""

# solution

print(ratings.head(), ratings.describe())

#----------------------------------#

# Conclusion

"""
Indeed! It looks like there are some faulty ratings (higher than 5) as the maximum is 19.
"""

                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend 

'\nIndeed! It looks like there are some faulty ratings (higher than 5) as the maximum is 19.\n'

In [38]:
X = ratings[['Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating']].dropna().iloc[:9367, :]

y = ratings['Rating'].dropna()

X.shape, y.shape

((9367, 6), (9367,))

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [40]:
def remove_non_numeric(column):
    """Removes non-numeric characters from a DataFrame column.

    Args:
        column (pd.Series): A pandas Series containing strings.

    Returns:
        pd.Series: A pandas Series with non-numeric characters removed.
    """

    return column.str.replace(r"[^\d\.]", "", regex=True).dropna()

X = X.apply(remove_non_numeric, axis=1)
X.replace("", 0, inplace=True)

/tmp/ipykernel_240251/2023531535.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.replace("", 0, inplace=True)


In [41]:
# exercise 02

"""
Predicting the rating of an app

Having explored the Google apps dataset in the previous exercise, let's now build a model that predicts the rating of an app given a subset of its features.

To do this, you'll use scikit-learn's DecisionTreeRegressor. As decision trees are the building blocks of many ensemble models, refreshing your memory of how they work will serve you well throughout this course.

We'll use the MAE (mean absolute error) as the evaluation metric. This metric is highly interpretable, as it represents the average absolute difference between actual and predicted ratings.

All required modules have been pre-imported for you. The features and target are available in the variables X and y, respectively.
"""

# Instructions

"""

    
    Use train_test_split() to split X and y into train and test sets. Use 20%, or 0.2, as the test size.
    
    Instantiate a DecisionTreeRegressor(), reg_dt, with the following hyperparameters: min_samples_leaf = 3 and min_samples_split = 9.
    
    Fit the regressor to the training set using .fit().
    
    Predict the labels of the test set using .predict().

"""

# solution

# Split into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the regressor
reg_dt = DecisionTreeRegressor(min_samples_leaf=3, min_samples_split=9, random_state=500)

# Fit to the training set
reg_dt.fit(X_train, y_train)

# Evaluate the performance of the model on the test set
y_pred = reg_dt.predict(X_test)
print('MAE: {:.3f}'.format(mean_absolute_error(y_test, y_pred)))

#----------------------------------#

# Conclusion

"""
Good job! As the MAE is around 0.61, the error of the model is, on average, less than one rating point away from the actual value! Having refreshed our understanding of how to build scikit-learn models, let's now jump into our first ensemble method: Voting!
"""

MAE: 0.455


"\nGood job! As the MAE is around 0.61, the error of the model is, on average, less than one rating point away from the actual value! Having refreshed our understanding of how to build scikit-learn models, let's now jump into our first ensemble method: Voting!\n"

In [2]:
# exercise 03

"""
Choosing the best model

In this exercise, you'll compare different classifiers and choose the one that performs the best.

The dataset here - already loaded and split into train and test sets - consists of Pokémon - their stats, types, and whether or not they're legendary. The objective of our classifiers is to predict this 'Legendary' variable.

Three individual classifiers have been fitted to the training set:

    clf_lr is a logistic regression.
    clf_dt is a decision tree.
    clf_knn is a 5-nearest neighbors classifier.

As the classes here are imbalanced - only 65 of the 800 Pokémon in the dataset are legendary - we'll use F1-Score to evaluate the performance. Scikit-learn's f1_score() has been imported for you.
"""

# Instructions

"""

    Predict the labels of X_test using each of the classifiers - clf_lr, clf_dt, and clf_knn.
---

    Calculate the F1-score for each of the classifiers. The f1_score() function takes in two arguments: the actual labels, y_test, and the classifier's predicted labels.
---
Question

Which model performs the best on the test set?
Possible answers:
    
    Logistic regression (clf_lr). {Answer}
    
    Decision tree (clf_dt).
    
    5-nearest neighbors (clf_knn).
"""

# solution

# Make the invidual predictions
pred_lr = clf_lr.predict(X_test)
pred_dt = clf_dt.predict(X_test)
pred_knn = clf_knn.predict(X_test)

# Evaluate the performance of each model
score_lr = f1_score(y_test, pred_lr)
score_dt = f1_score(y_test, pred_dt)
score_knn = f1_score(y_test, pred_knn)

# Print the scores
print(score_lr)
print(score_dt)
print(score_knn)

#----------------------------------#

# Conclusion

"""
Great choice! The logistic regression got the best F1-Score: ~59%. Therefore, it's the best individual choice!
"""

'\n\n'

In [3]:
# exercise 04

"""
Assembling your first ensemble

It's time to build your first ensemble model! The Pokémon dataset from the previous exercise has been loaded and split into train and test sets.

Your job is to leverage the voting ensemble technique using the sklearn API. It's up to you to instantiate the individual models and pass them as parameters to build your first voting classifier.
"""

# Instructions

"""

    
    Instantiate a KNeighborsClassifier called clf_knn with 5 neighbors (specified using n_neighbors).
    
    Instantiate a "balanced" LogisticRegression called clf_lr (specified using class_weight).
    
    Instantiate a DecisionTreeClassifier called clf_dt with min_samples_leaf = 3 and min_samples_split = 9.
    
    Build a VotingClassifier using the parameter estimators to specify the following list of (str, estimator) tuples: 'knn', clf_knn, 'lr', clf_lr, and 'dt', clf_dt.

"""

# solution

# Instantiate the individual models
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_lr = LogisticRegression(class_weight='balanced')
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)

# Create and fit the voting classifier
clf_vote = VotingClassifier(
    estimators=[('knn', clf_knn), ('lr', clf_lr), ('dt', clf_dt)]
)
clf_vote.fit(X_train, y_train)

#----------------------------------#

# Conclusion

"""
Congratulations - you just built your first ensemble model! It's now time to evaluate it.
"""

'\n\n'

In [4]:
# exercise 05

"""
Evaluating your ensemble

In the previous exercise, you built your first voting classifier. Let's now evaluate it and compare it to that of the individual models.

The individual models (clf_knn, clf_dt, and clf_lr) and the voting classifier (clf_vote) have already been loaded and trained.

Remember to use f1_score() to evaluate the performance. In addition, you'll create a classification report on the test set (X_test, y_test) using the classification_report() function.

Will your voting classifier beat the 58% F1-score of the decision tree?
"""

# Instructions

"""

    
    Use the voting classifier, clf_vote, to predict the labels of the test set, X_test.
    
    Calculate the F1-Score of the voting classifier.
    
    Calculate the classification report of the voting classifier by passing in y_test and pred_vote to classification_report().

"""

# solution

# Calculate the predictions using the voting classifier
pred_vote = clf_vote.predict(X_test)

# Calculate the F1-Score of the voting classifier
score_vote = f1_score(y_test, pred_vote)
print('F1-Score: {:.3f}'.format(score_vote))

# Calculate the classification report
report = classification_report(y_test, pred_vote)
print(report)

#----------------------------------#

# Conclusion

"""
Legendary! The performance is not higher, but is at least as good as the best individual model - the decision tree. This may be because the other two models had a performance around 10% lower than the decision tree.
"""

'\n\n'

In [43]:
got = pd.read_csv(path_data+'character-predictions.csv')
got.head()

,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
0,1,0,0,0.054,0.946,Viserys II Targaryen,NaN,1,NaN,NaN,...,0.0,NaN,0,0,NaN,11,1,1,0.605351,0
1,2,1,0,0.387,0.613,Walder Frey,Lord of the Crossing,1,Rivermen,208.0,...,NaN,1.0,1,1,97.0,1,1,1,0.896321,1
2,3,1,0,0.493,0.507,Addison Hill,Ser,1,NaN,NaN,...,NaN,NaN,0,1,NaN,0,0,0,0.267559,1
3,4,0,0,0.076,0.924,Aemma Arryn,Queen,0,NaN,82.0,...,NaN,0.0,1,1,23.0,0,0,0,0.183946,0
4,5,1,1,0.617,0.383,Sylva Santagar,Greenstone,0,Dornish,276.0,...,NaN,1.0,1,1,29.0,0,0,0,0.043478,1


In [45]:
# exercise 06

"""
Journey to Westeros

If you're a Game of Thrones fan, you might already know all about the fictional world of Westeros and the characters that inhabit it. Regardless, it's important to explore a new dataset before doing any modeling. That's what you'll do now!

The dataset is loaded into the environment and available to you as got, the commonly used acronym for Game of Thrones.

The target variable here is 'actual'. It represents whether a character is alive (1) or not (0). First explore the target using the .describe() method. What can you conclude about it?
"""

# Instructions

"""
Possible answers:
    
    The maximum value is 1946.
    
    There are no missing values. {Answer}
    
    Around 75% of the characters die.
"""

# solution

display(got.describe())

#----------------------------------#

# Conclusion

"""
You're right! It looks like there are no missing values in this column. Other features in the dataset do have some missing values, however.
"""

,S.No,actual,pred,alive,plod,male,dateOfBirth,DateoFdeath,book1,book2,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
count,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000,433.000000,444.000000,1946.000000,1946.000000,...,23.000000,276.000000,1946.000000,1946.000000,433.000000,1946.000000,1946.000000,1946.000000,1946.000000,1946.000000
mean,973.500000,0.745632,0.687050,0.634470,0.365530,0.619219,1577.364896,2950.193694,0.198356,0.374615,...,0.652174,0.778986,0.141829,0.460946,-1293.563510,0.305755,0.074512,0.059096,0.089584,0.745632
std,561.906131,0.435617,0.463813,0.312637,0.312637,0.485704,19565.414460,28192.245529,0.398864,0.484148,...,0.486985,0.415684,0.348965,0.498601,19564.340993,1.383910,0.262669,0.235864,0.160568,0.435617
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-298001.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,487.250000,0.000000,0.000000,0.391250,0.101000,0.000000,240.000000,282.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.013378,0.000000
50%,973.500000,1.000000,1.000000,0.735500,0.264500,1.000000,268.000000,299.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.033445,1.000000
75%,1459.750000,1.000000,1.000000,0.899000,0.608750,1.000000,285.000000,299.000000,0.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,50.000000,0.000000,0.000000,0.000000,0.086957,1.000000
max,1946.000000,1.000000,1.000000,1.000000,1.000000,1.000000,298299.000000,298299.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,100.000000,15.000000,1.000000,1.000000,1.000000,1.000000


"\nYou're right! It looks like there are no missing values in this column. Other features in the dataset do have some missing values, however.\n"

In [6]:
# exercise 07

"""
Predicting GoT deaths

While the target variable does not have any missing values, other features do. As the focus of the course is not on data cleaning and preprocessing, we have already done the following preprocessing for you:

    Replaced NA values with 0.
    Replace negative values of age with 0.
    Replace NA values of age with the mean.

Let's now build an ensemble model using the averaging technique. The following individual models have been built:

    Logistic Regression (clf_lr).
    Decision Tree (clf_dt).
    Support Vector Machine (clf_svm).

As the target is binary, all these models might have good individual performance. Your objective is to combine them using averaging. Recall from the video that this is the same as a soft voting approach, so you should still use the VotingClassifier().
"""

# Instructions

"""

    Set up the list of (string, estimator) tuples. Use 'lr' for clf_lr, 'dt' for clf_dt, and 'svm' for clf_svm.
    
    Build an averaging classifier called clf_avg. Be sure to specify an argument for the voting parameter.

"""

# solution

# Build the individual models
clf_lr = LogisticRegression(class_weight='balanced')
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf_svm = SVC(probability=True, class_weight='balanced', random_state=500)

# List of (string, estimator) tuples
estimators = [('lr',clf_lr),('dt',clf_dt),('svm',clf_svm)]

# Build and fit an averaging classifier
clf_avg = VotingClassifier(estimators, voting='soft')

clf_avg.fit(X_train, y_train)

# Evaluate model performance
acc_avg = accuracy_score(y_test,  clf_avg.predict(X_test))
print('Accuracy: {:.2f}'.format(acc_avg))

#----------------------------------#

# Conclusion

"""
Good job predicting the GoT deaths! With these three individual models, we could achieve a combined accuracy of ~80%!
"""

'\n\n'

In [7]:
# exercise 08

"""
Soft vs. hard voting

You've now practiced building two types of ensemble methods: Voting and Averaging (soft voting). Which one is better? It's best to try both of them and then compare their performance. Let's try this now using the Game of Thrones dataset.

Three individual classifiers have been instantiated for you:

    A DecisionTreeClassifier (clf_dt).
    A LogisticRegression (clf_lr).
    A KNeighborsClassifier (clf_knn).

Your task is to try both voting and averaging to determine which is better.
"""

# Instructions

"""

    
    Prepare the list of (string, estimator) tuples. Use 'dt' as the label for clf_dt, 'lr' for clf_lr, and 'knn' for clf_knn.
    
    Build a voting classifier called clf_vote.
    
    Build an averaging classifier called clf_avg.

"""

# solution

# List of (string, estimator) tuples
estimators = estimators = [('lr',clf_lr),('dt',clf_dt),('knn',clf_knn)]

# Build and fit a voting classifier
clf_vote = VotingClassifier(estimators)
clf_vote.fit(X_train, y_train)

# Build and fit an averaging classifier
clf_avg = VotingClassifier(estimators, voting='soft')
clf_avg.fit(X_train, y_train)

# Evaluate the performance of both models
acc_vote = accuracy_score(y_test, clf_vote.predict(X_test))
acc_avg = accuracy_score(y_test,  clf_avg.predict(X_test))
print('Voting: {:.2f}, Averaging: {:.2f}'.format(acc_vote, acc_avg))

#----------------------------------#

# Conclusion

"""
Well done, and congratulations on completing Chapter 1! It looks like 'Voting' was better to predict GoT deaths. When feasible, try both approaches and choose the better performing one. It's now time to learn about Bagging!
"""

'\n\n'